In [12]:
from elasticsearch import Elasticsearch

In [3]:
import sys
sys.path.append('/python/')
import connections as con

In [79]:
df = con.sql_to_dataframe('DISTRIBUTION', 'select device_id, pressure, temperature ,battery, timestamp from "DecentFrames"')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17250 entries, 0 to 17249
Data columns (total 6 columns):
id             17250 non-null int64
device_id      17250 non-null int64
pressure       17250 non-null float64
temperature    17250 non-null float64
battery        17250 non-null float64
timestamp      17250 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(3), int64(2)
memory usage: 808.7 KB


In [10]:
df.head()

,id,device_id,pressure,temperature,battery,timestamp
0,1,878,0.272217,17.3438,2.956,2018-08-29 10:14:54.806000+00:00
1,2,878,0.272156,17.3406,2.956,2018-08-29 10:16:54.758000+00:00
2,3,878,0.272095,17.3500,2.956,2018-08-29 10:17:56.404000+00:00
3,4,878,0.272186,17.3625,2.956,2018-08-29 10:18:58.610000+00:00
4,5,878,0.272156,17.3125,2.954,2018-08-29 10:19:56.195000+00:00


In [76]:
df["ts"] = df["timestamp"].ap

TypeError: <lambda>() got an unexpected keyword argument 'axis'

# ES

In [62]:
es = Elasticsearch([{'host': '178.128.199.231', 'port': 9200}])

In [63]:
mapping = {
  "mappings": {
    "vandstand": { 
      "properties": { 
        "device_id": { "type": "integer"  }, 
        "pressure": { "type": "float"  },
        "temperature": { "type": "float"  }, 
        "battery": { "type": "float"  },
        "timestamp": { 
            "type": "date",
            "format": "yyyy-MM-dd HH:mm:ssZ"
        }
      }
    }
  }
}

In [64]:
# ignore 400 cause by IndexAlreadyExistsException when creating an index
es.indices.create(index='klima', ignore=400, body=mapping)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'klima'}

In [67]:
def df_to_es(row):
    
    data = {
        "device_id": row['device_id'],
        "pressure": row['pressure'],
        "temperature": row['temperature'], 
        "battery": row['battery'],
        "timestamp": row['timestamp']
    }
    
    res = es.index(index='klima', doc_type='vandstand', body=data)

In [68]:
df.apply(df_to_es, axis=1)

TypeError: ("'Timestamp' object is not subscriptable", 'occurred at index 0')

In [61]:
es.indices.delete(index='klima', ignore=[400, 404])

{'acknowledged': True}